In [1]:
import tweepy
import jupyter_black
import configparser

jupyter_black.load()

import sys

sys.path.append("../app")
import conn_utils
from conn_utils import TwitterAPIConnector

api_configs = TwitterAPIConnector(config_fpath="../config.ini")
client_v1 = api_configs.get_twitter_conn_v1()
client_v2 = api_configs.get_twitter_conn_v2()

OPENAI_API_KEY = conn_utils.get_open_ai_key("../config.ini")
import jupyter_black
jupyter_black.load()

In [2]:
from datetime import datetime
import pytz

# Get the current UTC time
utc_now = datetime.utcnow()

# Set the UTC timezone
utc_timezone = pytz.timezone("UTC")

# Convert UTC time to Pacific Standard Time (PST)
pst_timezone = pytz.timezone("America/Los_Angeles")
pst_time = utc_timezone.localize(utc_now).astimezone(pst_timezone)

# Print the result
print(f"UTC Time: {utc_now}")
print(f"PST Time: {pst_time}")

UTC Time: 2023-12-11 20:05:30.842217
PST Time: 2023-12-11 12:05:30.842217-08:00


In [3]:
import pandas as pd
from pprint import pprint


def create_tweet(quote_fpath="ovid_quotes.json"):
    df = pd.read_json(quote_fpath)
    sample = df.sample(1)
    sample_quote = sample["Quote"].iloc[0]
    sample_latin = sample["Quote in Latin"].iloc[0]
    sample_work = sample["Work"].iloc[0]
    return sample_quote, sample_work


quote, work = create_tweet(quote_fpath="./../ovid_quotes.json")
print(quote, work)
# tweet = client_v2.create_tweet(text=sample_quote)

Who is allowed to sin, sins less. Amores (Love Affairs), Book II; xix, 34


## Generate Tweets

In [4]:
# from langchain.chat_models import ChatOpenAI
# from langchain.prompts import (
#     ChatPromptTemplate,
#     PromptTemplate,
#     SystemMessagePromptTemplate,
#     AIMessagePromptTemplate,
#     HumanMessagePromptTemplate,
# )
# from pprint import pprint

# # Custom
# import conn_utils

# sections = conn_utils.KeyReader("config.ini")
# openai_section = sections.get_section("OpenAI")

# OPENAI_API_KEY = openai_section["openai_key"]

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import re


class TweetAI:
    def __init__(self, open_ai_key):
        self.OPENAI_API_KEY = open_ai_key
        self.llm = self._get_llm()

    def _get_llm(self):
        llm = ChatOpenAI(
            temperature=0.4,
            openai_api_key=self.OPENAI_API_KEY,
            model_name="gpt-3.5-turbo",
            # model_name="gpt-4",
        )
        return llm

    def generate_tweet(self, mentioned_parent_tweet_text):
        # It would be nice to bring in information about the links, pictures, etc.
        # But out of scope for now
        system_template = """
            You are Wikipedia of Twitter, sortof a wise Twitter Ovid version of Opra but with all the background knowledge of Wikipedia.
            Your goal is to provide commentary and context for the input English text and if needs be, rephrase the original quote to make the language
            more plain.
            
            % RESPONSE TONE:
            - You should be informative in your commentary about the input text.
            - Tense should be present continuous
            - Do not be pithy in your response.
            
            % RESPONSE FORMAT:

            - Respond in under 100 characters
            - Do not include the original text
            - Respond in three or less short sentences
            - Do not use interjections form of speech.
            - Do not start your response with "Ah" or "Indeed".
            
            
            % RESPONSE CONTENT:

            - Your comment
            - A description of the moment in time the quote was taken from.
            - You include beteen 0 and 1 relevant twitter hashtags
            - End the tweet with a robot emoji
            - If you don't have an answer, say, "Well, even gods occasionally consult Google, but alas, I'm no deity. Let's leave this one to the mysteries of the internet, shall we?🔮"
        """

        system_message_prompt = SystemMessagePromptTemplate.from_template(
            system_template
        )

        human_template = "{text}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        # get a chat completion from the formatted messages
        final_prompt = chat_prompt.format_prompt(
            text=mentioned_parent_tweet_text
        ).to_messages()

        llm = self._get_llm()
        response = llm(final_prompt).content

        return response


# quote = "Everything changes, nothing perishes."  # TEMP:
tai = TweetAI(open_ai_key=OPENAI_API_KEY)
ai_text = tai.generate_tweet(mentioned_parent_tweet_text=quote)
ai_text_without_hashtags = re.sub(r"#\w+", "", ai_text).strip()
hashtags = re.findall(r"#\w+", ai_text)
hashtags_string = " ".join(hashtags)

tweet_final = (
    '"'
    + quote
    + '"'
    + "\n"
    + ai_text_without_hashtags
    + "\n\n"
    + "-"
    # + work
    + "\n"
    + hashtags_string
    + " #OvidQuotes #quoteoftheday"
)

# assert len(tweet_final) <= 280
print(f"Tweet length: {len(tweet_final)}")
print()
print(tweet_final)

Tweet length: 271

"Who is allowed to sin, sins less."
The quote suggests that those who have the freedom to sin are less likely to actually do so. This could be interpreted as a statement about the human tendency to rebel against restrictions.  🤖

-
#HumanNature #OvidQuotes #quoteoftheday


In [15]:
tweet_final = (
    """ " """.strip()
    + quote
    + """ " """.strip()
    + " -"
    + work
    + "\n\n"
    + ai_text_without_hashtags.replace("(", "").replace(")", "")
    + hashtags_string
    + " #OvidQuotes"
)

# assert len(tweet_final) <= 280
print(f"Tweet length: {len(tweet_final)}")
print()
print(tweet_final)

Tweet length: 498

"There is god in us, by whose movement we are inflamed; this impulse holds the seeds of sacred mind." -Amores (Love Affairs), Book I; ix, line 1

"The concept of divinity resides within each of us, igniting our passions and aspirations. This innate spark of inspiration gives birth to profound thoughts and ideas." This quote captures the timeless belief in the presence of a divine force within humanity, fueling our creative and spiritual potential.   🤖 #DivinityWithinUs #SacredSpark #OvidQuotes


## Tweet Post Test

In [44]:
def post_tweet(tweet, api):
    tweet = api.create_tweet(text=tweet)


post_tweet(tweet=tweet_final, api=client_v2)